- Nombre: Alejandro Velazco 
1. Busca y descarga el conjunto de datos “Climate Model Simulation Crashes” del repositorio UCI Machine Learning. Tiene los datos de un problema de clasificación.
2. Cárgalo adecuadamente en un Pandas DataFrame.
3. Realiza un conteo de las clases y analiza si hay desequilibrio.
4. Evalua el rendimiento del modelo k-Nearest Neigbours mediante las
tres estrategias de validación cruzada estudiados y utilizando la
métrica que consideres más adecuada.

In [17]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import  mean_squared_error
from sklearn.model_selection import KFold, LeaveOneOut, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

In [30]:

# 1. Descargar el conjunto de datos
url = "./pop_failures.dat"

data = pd.read_csv(url, delimiter=r"\s+")  # El archivo tiene 21 líneas de metadatos al principio

# 2. Cargarlo en un Pandas DataFrame
df = data.copy()

# 3. Conteo de clases y análisis de desequilibrio
print(df['outcome'].value_counts())  # 'outcome' es la columna de etiquetas

# 4. Evaluar el rendimiento de k-Nearest Neighbors

# Separar las características y etiquetas
X = df.drop(['outcome'], axis=1).values
y = df['outcome'].values

# Normalizar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Crear modelo KNN
knn = KNeighborsClassifier(n_neighbors=5)

# Estrategias de validación cruzada
kf = KFold(n_splits=10)
loocv = LeaveOneOut()

# Función para convertir RMSE a Accuracy
def rmse_to_accuracy(rmse_values):
    accuracies = [np.sqrt(abs(error)) for error in rmse_values]
    return np.mean(accuracies)

# KFold
kf_rmse_scores = cross_val_score(knn, X_scaled, y, cv=kf, scoring='neg_root_mean_squared_error')
print(f"KFold Adjusted Accuracy: {rmse_to_accuracy(kf_rmse_scores):.4f}")

# LOOCV
loocv_rmse_scores = cross_val_score(knn, X_scaled, y, cv=loocv, scoring='neg_root_mean_squared_error')
print(f"LOOCV Adjusted Accuracy: {rmse_to_accuracy(loocv_rmse_scores):.4f}")

# Validación cruzada Monte Carlo
monte_carlo_scores = []
for _ in range(100):  # Realizar 100 iteraciones/splits aleatorios
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3)
    knn.fit(X_train, y_train)
    predictions = knn.predict(X_test)
    mse_value = mean_squared_error(y_test, predictions)
    rmse_value = np.sqrt(mse_value)
    monte_carlo_scores.append(-rmse_value)  # Agregar el RMSE negativo para que sea consistente con las otras estrategias

print(f"Monte Carlo Adjusted Accuracy: {rmse_to_accuracy(monte_carlo_scores):.4f}")

outcome
1    494
0     46
Name: count, dtype: int64
KFold Adjusted Accuracy: 0.5248
LOOCV Adjusted Accuracy: 0.0796
Monte Carlo Adjusted Accuracy: 0.5257
